<a href="https://colab.research.google.com/github/UltimateWing/WUG.Python.ShioajiApiLogic/blob/main/ShioajiApiFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install shioaji

In [1]:
import shioaji as sj

In [18]:
class ShiojiApiLogic():
  #建構式
  def __init__(self, isSimulation: bool = True):
    self.IsSimulation = isSimulation
    if isSimulation:
      self.Api = sj.Shioaji(simulation = True)
    else:
      self.Api = sj.Shioaji()
      self.LoginId = ""
      self.IsConnected = False
      self.AccountList = []
      self.ReplyDict = {}
    
  #模擬環境初始化
  def InitSimulation(self, virtualIdNum: int = 1):
    virtualId = "PAPIUSER0" + str(virtualIdNum)
    pwd = "2222"
    self.Init(virtualId, pwd)

  #正式環境初始化
  def Init(self, loginId: str, loginPwd: str, caPath: str = "", caPwd: str = ""):
    self.Api.login(
        person_id = loginId,
        passwd = loginPwd,
        contracts_cb = lambda security_type: print(f"{repr(security_type)} fetch done.")
        )
    self.AccountList = self.Api.list_accounts()
    #註冊回報事件
    self.Api.set_order_callback(self.__placeCallback)
    if self.IsSimulation == False:
      result = self.Api.activate_ca(
          ca_path = caPath,
          ca_passwd = caPwd,
          person_id = loginId
          )
      
  #委託下單
  def SendOrder(
      self, symbolId: str, orderPrice: float, orderQty: int, 
      bsType: str, priceType: str, timeInForce: str,
      sessionType: str, isFirstSell: bool, accountType: str):
    contract = self.__getContract(symbolId, accountType)    
    #使用預設帳號下單
    order = self.Api.Order(
        price = orderPrice,
        quantity = orderQty,
        action = self.__getAction(bsType),
        price_type = self.__getPriceType(priceType),
        order_type = self.__getTimeInForce(timeInForce),                      
        order_lot = self.__getSessionType(sessionType),
        first_sell = self.__getFirstSellType(isFirstSell),
        account = self.__getAccountInfo(accountType)
        )
    trade = self.Api.place_order(contract, order)
    return trade
    
  #def CancelOrder(self, orderNo: str)
    
    
  #登出
  def Logout(self):
    self.Api.logout()
    
    
  #Private Method
    
  #取得買賣類型
  def __getAction(self, bsType: str):
    if bsType == "Buy":
      return sj.constant.Action.Buy
    elif bsType == "Sell":
      return sj.constant.Action.Sell
    else:
      raise Exception(
          "Not correct Action: " + bsType + 
          ", only can input {Buy: 買, Sell: 賣}"
          )
            
  #取得價格類型
  def __getPriceType(self, priceType: str):
    if priceType == "Limit": #限價
      return sj.constant.StockPriceType.LMT
    elif priceType == "Market": #市價
      return sj.constant.StockPriceType.MKT
    elif priceType == "RangeMarket": #範圍市價
      return sj.constant.StockPriceType.MKP
    else:
      raise Exception(
          "Not correct StockPriceType: " + priceType + 
          ", only can input {Limit: 限價, Market: 市價, RangeMarket: 範圍市價}")

  #取得委託單時效性
  def __getTimeInForce(self, timeInForce: str):
    if timeInForce == "ROD":
      return sj.constant.TFTOrderType.ROD
    elif timeInForce == "IOC":
      return sj.constant.TFTOrderType.IOC
    elif timeInForce == "FOK":
      return sj.constant.TFTOrderType.FOK
    else:
      raise Exception(
          "Not correct TimeInForce: " + timeInForce +
          ", only can input {ROD, IOC, FOK}")
            
  #取得委託單盤別
  def __getSessionType(self, sessionType: str):
    if sessionType == "RegularTrading":
      return sj.constant.TFTStockOrderLot.Common
    elif sessionType == "AfterHoursTrading":
      return sj.constant.TFTStockOrderLot.Fixing
    elif sessionType == "OddLot":
      return sj.constant.TFTStockOrderLot.Odd
    elif sessionType == "OddLotAtSession":
      return sj.constant.TFTStockOrderLot.IntradayOdd
    else:
      raise Exception(
          "Not correct SessionType: " + sessionType +
          ", only can input {RegularTrading: 整股, " +
                     "AfterHoursTrading: 定盤, " +
                     "OddLot: 盤後零股, " + 
                     "OddLotAtSession: 盤中零股}"
          )

  def __getFirstSellType(self, isFirstSell: bool):
    if isFirstSell:
      return "true"
    else:
      return "false"

  #取得帳號資訊  
  def __getAccountInfo(self, accountType):
    if accountType == "Stk":
      return self.Api.stock_account
    elif accountType == "Fut":
      return self.Api.futopt_account
    else:
      raise Exception(
          "Not correct AccountType: " + accountType +
          ", only can input {Stk: 現貨帳戶資訊, Fut: 期貨帳戶資訊}"
      )

  #依帳戶類別取得商品合約，帳戶有股票和期貨，
  #以此為依據去取得Shioaji資料庫中Contracts.Stocks或Contracts.Futures的合約資料結構
  def __getContract(self, symbolId: str, accountType: str):
    if accountType == "Stk":
      return self.Api.Contracts.Stocks[symbolId]
    elif accountType == "Fut":
      return self.Api.Contracts.Futures[symbolId]
    else:
      raise Exception(
          "Not correct AccountType:" + accountType + 
          ", and then cause cannot catch contract. " +
          "Only can input {Stk: 現貨帳戶資訊, Fut: 期貨帳戶資訊}")
    
  #Event
    
  #接收委託回報
  def __placeCallback(stat, msg):
    print('my_place_callback')
    print('stat=', stat)
    print('----------')
    print('msg=',msg)        
    #if msg['order']['id']

In [19]:
#測試執行
Api = ShiojiApiLogic(isSimulation=True)
Api.InitSimulation(1)

Response Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.


In [11]:
Api.AccountList

[FutureAccount(person_id='QBCCAIGJBJ', broker_id='F002000', account_id='9100020', signed=True, username='PAPIUSER01'),
 StockAccount(person_id='QBCCAIGJBJ', broker_id='9A95', account_id='0504350', signed=True, username='PAPIUSER01')]

In [20]:
Api.SendOrder("2886", 30, 1, "Buy", "Limit", "ROD", "RegularTrading", False, "Stk")

<SecurityType.Stock: 'STK'> fetch done.
<SecurityType.Option: 'OPT'> fetch done.


Trade(contract=Stock(exchange=<Exchange.TSE: 'TSE'>, code='2886', symbol='TSE2886', name='兆豐金', category='17', unit=1000, limit_up=36.6, limit_down=30.0, reference=33.3, update_date='2021/11/03', margin_trading_balance=42520, short_selling_balance=21, day_trade=<DayTrade.Yes: 'Yes'>), order=Order(action=<Action.Buy: 'Buy'>, price=30, quantity=1, account=StockAccount(person_id='QBCCAIGJBJ', broker_id='9A95', account_id='0504350', signed=True, username='PAPIUSER01'), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>), status=OrderStatus(status=<Status.Inactive: 'Inactive'>))

In [17]:
Api.Logout()

In [ ]:
def InitSimulation(userId : int = 1):
    name = "PAPIUSER0" + str(userId)
    pwd = "2222"
    api = sj.Shioaji(simulation=True)    
    api.login(person_id="PAPIUSER01",
          passwd="2222",
          contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
          )

In [ ]:
#取得帳號清單
accounts = api.list_accounts()

In [ ]:
#取得預設帳戶
stkAcc = api.stock_account
futAcc = api.futopt_account

In [ ]:
def SendOrder(symbolId, orderPrice, orderQty, bsType, priceType, timeInForce, sessionType, isFirstSell, accountType):
    contract = __getContract(symbolId, accountType)    
    #使用預設帳號下單
    order = api.Order(
                      price = orderPrice,
                      quantity = orderQty,
                      action = __getAction(bsType),
                      price_type = __getPriceType(priceType),
                      order_type = __getTimeInForce(timeInForce),                      
                      order_lot = __getSessionType(sessionType),
                      first_sell = isFirstSell,
                      account = __getAccountInfo(accountType)
                     )
    trade = api.place_order(contract, order)
    return trade

In [ ]:
SendOrder("2886", 10, 1, "Buy", "Limit", "ROD", "RegularTrading", False, "Stk")

AttributeError: 'NoneType' object has no attribute 'Stocks'

In [ ]:
#取得買賣類型
def __getAction(bsType):
    if bsType == "Buy":
        return sj.constant.Action.Buy
    elif bsType == "Sell":
        return sj.constant.Action.Sell
    else:
        raise Exception("Not correct Action: " + bsType + ", only can input {Buy: 買, Sell: 賣}")

In [ ]:
GetAction("")

Exception: Not correct Action: , only can input {B: 買, S: 賣}

In [ ]:
#取得價格類型
def __getPriceType(priceType):
    if priceType == "Limit": #限價
        return sj.constant.StockPriceType.LMT
    elif priceType == "Market": #市價
        return sj.constant.StockPriceType.MKT
    elif priceType == "RangeMarket": #範圍市價
        return sj.constant.StockPriceType.MKP
    else:
        raise Exception("Not correct StockPriceType: " + priceType + 
                        ", only can input {Limit: 限價, Market: 市價, RangeMarket: 範圍市價}")

In [ ]:
GetPriceType("LM12T")

Exception: Not correct StockPriceType: LM12T, only can input {LMT: 限價, MKT: 市價, MKP: 範圍市價}

In [ ]:
#取得委託單時效性
def __getTimeInForce(timeInForce):
    if timeInForce == "ROD":
        return sj.constant.TFTOrderType.ROD
    elif timeInForce == "IOC":
        return sj.constant.TFTOrderType.IOC
    elif timeInForce == "FOK":
        return sj.constant.TFTOrderType.FOK
    else:
        raise Exception("Not correct TimeInForce: " + timeInForce +
                        ", only can input {ROD, IOC, FOK}")

In [ ]:
GetTimeInForce("FOK")

<TFTOrderType.FOK: 'FOK'>

In [ ]:
#取得委託單盤別
def __getSessionType(sessionType):
    if sessionType == "RegularTrading":
        return sj.constant.TFTStockOrderLot.Common
    elif sessionType == "AfterHoursTrading":
        return sj.constant.TFTStockOrderLot.Fixing
    elif sessionType == "OddLot":
        return sj.constant.TFTStockOrderLot.Odd
    elif sessionType == "OddLotAtSession":
        return sj.constant.TFTStockOrderLot.IntradayOdd
    else:
        raise Exception("Not correct SessionType: " + sessionType +
                        ", only can input {RegularTrading: 整股, " +
                                          "AfterHoursTrading: 定盤, " +
                                          "OddLot: 盤後零股, " + 
                                          "OddLotAtSession: 盤中零股}")

In [ ]:
#取得帳戶資訊
def __getAccountInfo(accountType):
    if accountType == "Stk":
        return api.stock_account
    elif accountType == "Fut":
        return api.futopt_account
    else:
        raise Exception("Not correct AccountType: " + accountType +
                        "only can input {Stk: 取得API預設現貨帳戶資料, Fut: 取得API預設期貨帳戶資料}")

In [ ]:
#依帳戶類別取得商品合約，帳戶有股票和期貨，以此為依據去取得Shioaji資料庫中Contracts.Stocks或Contracts.Futures的合約資料結構
def __getContract(symbolId, accountType):
    if accountType == "Stk":
        return api.Contracts.Stocks[symbolId]
    elif accountType == "Fut":
        return api.Contracts.Futures[symbolId]
    else:
        raise Exception("Not correct AccountType:" + accountType + ", and then cause cannot catch contract")

In [ ]:
#登出
def Logout():
    api.logout()